In [1]:
from pathlib import Path
import geopandas as gpd

from flood_adapt.api.static import read_database
from flood_adapt.misc.config import Settings, UnitSystem

In [2]:
db_path = Path(r"C:\Users\tromp_wm\fa_databases\scheveningen")
units = UnitSystem(system="metric")


settings = Settings(
    DATABASE_ROOT=db_path.parent,
    DATABASE_NAME=db_path.stem,
    SYSTEM_FOLDER=db_path/"system",
    unit_system=units
)

db = read_database(
    database_path=db_path.parent,
    site_name=db_path.stem
)

In [ ]:
db

52.09835944139894

In [ ]:
from flood_adapt.api.scenarios import get_scenarios

In [ ]:
scenarios = get_scenarios()
scenarios

In [ ]:
scenarios["objects"][0].impacts.impacts_path

In [ ]:
scenarios["objects"][0].attrs.name

In [ ]:
[scen.attrs.name for scen in scenarios["objects"] if scen.has_run_check()]

In [ ]:
aggs = gpd.read_file(scenarios["objects"][0].impacts.impacts_path/"Impacts_aggregated_current_no_measure_region.gpkg")
builds = gpd.read_file(scenarios["objects"][0].impacts.impacts_path/"Impacts_building_footprints_current_no_measure.gpkg")

In [ ]:
builds

In [ ]:
aggs

In [ ]:
m = builds.explore(column="Total Damage %", cmap="YlOrRd", legend=True, style_kwds={"color": "black"}, scheme='NaturalBreaks', k=10, legend_kwds={"colorbar": False})

In [ ]:
import leafmap
from ipyleaflet import basemaps, LegendControl

In [ ]:
m = leafmap.Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

In [ ]:
m

In [ ]:
m.add_data(
    builds,
    column="Total Damage",
    cmap="YlOrRd",
    scheme="NaturalBreaks",
    info_mode="on_hover",
    add_legend=False,
    style={
        "stroke": True,
        "color": "black",
        "weight": 1,
        "opacity": 1,
        "fillOpacity": 1,
    },
)

In [ ]:
from mapclassify import NaturalBreaks

nb5 = NaturalBreaks(aggs["TotalDamageEvent"], k=5)

In [ ]:
nb5

In [ ]:
import mapclassify

In [ ]:
legend_keys = nb5.get_legend_classes()

In [ ]:
legend_keys

In [ ]:
import matplotlib as mpl
import numpy as np

cmap = mpl.colormaps["YlOrRd"]

In [ ]:
legend_values = cmap(np.linspace(0,1,5))

In [ ]:
lgnd = {legend_keys[i]:mpl.colors.rgb2hex(legend_values[i]) for i in range(0,5)}

In [ ]:
legend_control = LegendControl(lgnd)

In [ ]:
m.add(legend_control)

In [ ]:
m.controls

In [ ]:
from ipyleaflet import WidgetControl, Output

for control in m.controls:
    if isinstance(control, WidgetControl):
        if isinstance(control.widget, Output):
            control.widget.layout.overflow="auto"
            control.position="topright"
            print(control.widget.trait_names())


In [ ]:
from flood_adapt.object_model.interface.measures import MeasureType

In [ ]:
[e.value for e in MeasureType]

In [ ]:
settings.unit_system.length

In [ ]:
Settings().unit_system.length

In [ ]:
db.site.attrs.gui.units.default_length_units

In [ ]:
from flood_adapt.dbs_classes.database import Database

from flood_adapt.object_model.io import unit_system as us

In [ ]:
Database().site.attrs.gui.units

In [ ]:
int_unit = Database().site.attrs.gui.units.default_intensity_units

In [ ]:
print(int_unit)

In [ ]:
isinstance({"value": 20, "units": int_unit.value}, us.UnitfulIntensity)

In [ ]:
us.UnitfulIntensity(
    value=10,
    units=int_unit
)

In [ ]:
from flood_adapt.object_model.hazard.interface.events import Template, IEventModel

In [ ]:
for e in Template:
    print(f"{e.value}: {e.description}")

In [ ]:
IEventModel.model_fields

In [ ]:
from flood_adapt.object_model.hazard.event.synthetic import SyntheticEventModel
from flood_adapt.object_model.hazard.event.historical import HistoricalEventModel, HistoricalEvent

In [ ]:
HistoricalEvent._attrs_type

In [ ]:
SyntheticEventModel.ALLOWED_FORCINGS

In [ ]:
for forcing in SyntheticEventModel.ALLOWED_FORCINGS:
    print(forcing.name)

In [ ]:
from flood_adapt.object_model.hazard.event.event_factory import EventFactory

In [ ]:
etype = "Synthetic"
forcings = EventFactory._EVENT_TEMPLATES[etype][1].ALLOWED_FORCINGS

In [ ]:
forcings["RAINFALL"][0]

In [ ]:
from flood_adapt.object_model.io.unit_system import UnitTypesTime

In [ ]:
UnitTypesTime.hours

In [ ]:
lst = ["a", "b", "c"]

In [ ]:
new = [*lst.pop(1)]

In [ ]:
new

In [ ]:
Database().site.attrs.sfincs.river

In [ ]:
from ipyleaflet import Map, GeomanDrawControl, basemaps, LayersControl, LegendControl
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
feats = []

def handle_draw(self, action, geo_json):
    print(self)
    print(action)
    print(geo_json)
    feats.append(self)

m = Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik, interpolation='nearest')
draw_control = GeomanDrawControl()
draw_control.on_draw(handle_draw)
draw_control.polyline = {}
m.add(draw_control)
m.add(LayersControl())

In [ ]:
import rioxarray
import rasterio
from localtileserver import TileClient, get_leaflet_tile_layer
import xarray as xr

In [ ]:
fn = r"C:\Users\tromp_wm\fa_databases\scheveningen\output\scenarios\current_no_measure\Flooding\FloodMap_current_no_measure.tif"
base_fn = r"C:\Users\tromp_wm\fa_databases\scheveningen\static\dem\dep_subgrid.tif"

In [ ]:
da = xr.open_dataarray(fn)
base = xr.open_dataarray(base_fn)
da = da.sel(band=1)
base = base.sel(band=1)
# da = rasterio.open(fn)

In [ ]:
da = da.where(base>0)

In [ ]:
da = da.rio.reproject(4326)
da.name = "floodmap"

In [ ]:
mem_file = rasterio.MemoryFile()
raster_dataset = mem_file.open(
    driver="GTiff",
    height=da.shape[0],
    width=da.shape[1],
    count=1,
    dtype=da.dtype,
    crs=da.rio.crs,
    transform=da.rio.transform(),
    nodata=np.nan,
)
raster_dataset.write(da.data, 1)
raster_dataset.close()

In [ ]:
client = TileClient(raster_dataset)

In [ ]:
bupu = mpl.colormaps["BuPu"]
colors = bupu(np.linspace(0,1,bupu.N))
cmap = mpl.colors.LinearSegmentedColormap.from_list("cmap", colors, N=bupu.N)

In [ ]:
{k:mpl.colors.to_hex(v) for k,v in enumerate(colors)}

In [ ]:
cmap

In [ ]:
vmin = 0
vmax = 2
legend_percs = [0.25,0.5,0.75,1]
# legend_keys = {(perc*(vmax-vmin)+vmin):list(np.roll(mpl.colors.to_hex(cmap(perc)),1)) for perc in legend_percs}
legend_keys = [perc*(vmax-vmin)+vmin for perc in legend_percs]
legend_values = [mpl.colors.to_hex(cmap(perc)) for perc in legend_percs]
legend_values = list(np.roll(legend_values,1))
legend = dict(zip(legend_keys,legend_values))

In [ ]:
legend

In [ ]:
t = get_leaflet_tile_layer(client, colormap=cmap, vmin=vmin, vmax=vmax)

In [ ]:
m.add(t)

In [ ]:
m.controls

In [ ]:
legend_control = LegendControl(
    legend,
    name="water level",
    position="topright"
)
m.add(legend_control)

In [ ]:
legend_control.legend

In [ ]:
floodlayer = da.leaflet.plot(m, colormap = plt.cm.terrain)
# floodlayer.interact(opacity=(0.0,1.0,0.1))
# m.add(floodlayer)



In [ ]:
floodlayer

In [ ]:
m.layers

In [ ]:
m.add(LayersControl())

In [ ]:
m.add(l)

In [ ]:
m.bounds

In [ ]:
for feat in feats:
    feat.clear()

In [ ]:
def remove_control(m):
    for control in m.controls:
        if isinstance(control, GeomanDrawControl):
            m.remove(control)

In [ ]:
remove_control(m)

In [ ]:
m.controls

In [ ]:
for control in m.controls:
    if isinstance(control, GeomanDrawControl):
        print("adding handle draw")
        control.on_draw(handle_draw)

In [ ]:
m

In [ ]:
feats

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
file_fn = Path(r"C:\Users\tromp_wm\fa_databases\scheveningen\input\measures\testwall\selection.geojson")
wall = gpd.read_file(file_fn)

In [ ]:
wall

In [ ]:
wall.plot()

In [ ]:
import xarray as xr
import numpy as np

In [ ]:
flood_fn = r"C:\Users\tromp_wm\fa_databases\scheveningen\output\scenarios\current_no_measure\Flooding\FloodMap_current_no_measure.tif"
base_fn = r"C:\Users\tromp_wm\fa_databases\scheveningen\static\dem\dep_subgrid.tif"

In [ ]:
flood = xr.open_dataarray(flood_fn)
base = xr.open_dataarray(base_fn)

vmin=0
vmax=2
N=5
bins = [*np.linspace(vmin,vmax,N), np.inf]

In [ ]:
bins

In [ ]:
base = base.sel(band=1)
flood = flood.where(base>0)

In [ ]:
flood_grp = flood.groupby_bins(flood,bins)
def minim(x):
    return (x-x+x.min(...))
flood_binned = flood_grp.apply(minim).sortby([flood.x, flood.y])

In [ ]:
flood_binned.sel(band=1)

In [ ]:
import leafmap.leafmap as leafmap
from ipyleaflet import basemaps

In [ ]:
m = leafmap.Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

In [ ]:
m

In [ ]:
m.add_raster(flood_binned, colormap="Blues", vmin=vmin, vmax=vmax, nodata=np.nan)